# Analysis of Ofsted children's home inspections

This notebook analyzes inspection data from the UK's Ofsted agency, which regulates children's homes. Please [read this page](https://github.com/BuzzFeedNews/2018-07-ofsted-inspections) for more details and context.

## Load the private-sector ownership data

In [1]:
import pandas as pd

In [2]:
private_ownerships = pd.read_csv(
    "../inputs/private-sector-ownerships.csv",
    dtype = str
)

In [3]:
CAMBIAN_PATTERN = r"Cambian|Interact Care"

## Load the "as at 31 March 2018" data

... and merge with ownership data.

In [4]:
as_at_data = (
    pd.read_csv(
        "../inputs/providers-as-at-2018-03-31.csv",
        skiprows = 1,
        dtype = str
    )
    .assign(**{
        "Inspection date": lambda df: pd.to_datetime(df["Inspection date"], format = "%d/%m/%Y"),
        "Publication date": lambda df: pd.to_datetime(df["Publication date"], format = "%d/%m/%Y"),
    })
    [[
        "URN",
        "Provision type",
        "Registration status",
        "Sector",
        "Event type",
        "Inspection date",
        "Publication date",
        "Overall effectiveness"
    ]]
       
    # Add ownership information
    .merge(
        private_ownerships[[
            "URN",
            "Owner"
        ]],
        how = "left"
    )
    
    # Classify private-sector ownership
    .assign(
        owned_by_cambian = lambda x: x["Owner"].str.contains(CAMBIAN_PATTERN, na = False, case = False)    
    )
)

as_at_data.head()

,URN,Provision type,Registration status,Sector,Event type,Inspection date,Publication date,Overall effectiveness,Owner,owned_by_cambian
0,1027076,Secure Training Centre,Active,Private,Full inspection,2016-06-24,2016-08-08,Inadequate,NaN,False
1,1027077,Secure Training Centre,Active,Private,Full inspection,2017-10-06,2017-11-21,Inadequate,NaN,False
2,1027078,Secure Training Centre,Active,Private,Full inspection,2017-06-23,2017-08-08,Requires improvement to be good,NaN,False
3,1027081,Residential Holiday Scheme for Disabled Children,Active,Voluntary,Full inspection,2017-08-24,2017-09-15,Good,NaN,False
4,1027082,Residential Holiday Scheme for Disabled Children,Active,Voluntary,Full inspection,2017-08-16,2017-09-11,Outstanding,NaN,False


### Examine the `Registration status`, `Provision type`, `Overall effectiveness`, and `Sector` variables

In [5]:
as_at_data["Registration status"].value_counts()

Active    2797
Name: Registration status, dtype: int64

In [6]:
as_at_data["Provision type"].value_counts()

Children's home                                                 2038
Independent Fostering Agency                                     278
Residential Special School                                       150
Boarding School                                                   77
Residential special school (registered as a children's home)      71
Voluntary Adoption Agency                                         40
Further Education College with Residential Accommodation          38
Residential Family Centre                                         37
Adoption Support Agency                                           34
Residential Holiday Scheme for Disabled Children                  16
Secure children's home                                            14
Secure Training Centre                                             3
CAFCASS                                                            1
Name: Provision type, dtype: int64

In [7]:
as_at_data["Overall effectiveness"].value_counts()

Good                               1759
Outstanding                         548
Requires improvement to be good     399
Inadequate                           60
Requires improvement                 27
Adequate                              3
Satisfactory                          1
Name: Overall effectiveness, dtype: int64

In [8]:
as_at_data["Sector"].value_counts()

Private             1876
Local Authority      491
Voluntary            358
Academy               61
Health Authority      10
Public sector          1
Name: Sector, dtype: int64

## Apply data filters

In [9]:
as_at_data_filtered = (
    as_at_data
    # Most recent listed inspection on/after 1 April 2016
    .loc[lambda df: df["Inspection date"] >= "2016-04-01"]  

    # Filter on provision type
    .loc[lambda df: df["Provision type"].isin([
        "Children's home",
        "Residential special school (registered as a children's home)"
    ])]

    # Exclude the few homes run by health authorities
    .loc[lambda x: x["Sector"] != "Health Authority"]
    
    # Standardize "Requires improvement" phrasing
    .replace({ "Overall effectiveness": {
        "Requires improvement": "Requires improvement to be good"
    }})
)

len(as_at_data_filtered)

2083

In [10]:
# Make sure that we've identified all private-sector owners
assert (
    as_at_data_filtered
    .loc[lambda df: df["Sector"] == "Private"]
    ["Owner"].isnull()
    .sum()
) == 0

In [11]:
as_at_data_filtered["Sector"].value_counts()

Private            1529
Local Authority     401
Voluntary           153
Name: Sector, dtype: int64

In [12]:
as_at_data_filtered["Overall effectiveness"].value_counts()

Good                               1354
Requires improvement to be good     352
Outstanding                         346
Inadequate                           31
Name: Overall effectiveness, dtype: int64

Cambian-owned homes, by specific owner name in Ofsted data:

In [13]:
(
    as_at_data_filtered
    .loc[lambda x: x["owned_by_cambian"]]
    ["Owner"]
    .value_counts()
)

Cambian Childcare Ltd                         149
Cambian Autism Services Ltd                     4
Interact Care Limited                           2
Cambian Asperger Syndrome Services Limited      1
Cambian Signpost Limited                        1
Cambian Autism Services Limited                 1
Cambian Whinfell School Ltd                     1
Name: Owner, dtype: int64

## Analyze "as at" data

In [14]:
def calculate_totals(grouped_df, normalize = False):
    return (
        grouped_df
        ["Overall effectiveness"]
        .value_counts(normalize = normalize)
        .unstack()
        .fillna(0)
        [[
            "Inadequate",
            "Requires improvement to be good",
            "Good",
            "Outstanding",
        ]]
        .assign(
            goodplus = lambda df: df[[ "Good", "Outstanding" ]].sum(axis = 1),
            subpar = lambda df: df[[ "Inadequate", "Requires improvement to be good" ]].sum(axis = 1),
            total = lambda df: df["goodplus"] + df["subpar"]        
        )
    )

### By sector

In [15]:
(
    as_at_data_filtered
    .groupby([
        "Sector",
    ])
    .pipe(calculate_totals)
)

Overall effectiveness,Inadequate,Requires improvement to be good,Good,Outstanding,goodplus,subpar,total
Sector,,,,,,,
Local Authority,4,57,239,101,340,61,401
Private,26,272,1014,217,1231,298,1529
Voluntary,1,23,101,28,129,24,153


### By sector and ownership

In [16]:
(
    as_at_data_filtered
    .groupby([
        "Sector",
        "owned_by_cambian"
    ])
    .pipe(calculate_totals)
)

Overall effectiveness             Inadequate  Requires improvement to be good  \
Sector          owned_by_cambian                                                
Local Authority False                      4                               57   
Private         False                     23                              236   
                True                       3                               36   
Voluntary       False                      1                               23   

Overall effectiveness             Good  Outstanding  goodplus  subpar  total  
Sector          owned_by_cambian                                              
Local Authority False              239          101       340      61    401  
Private         False              907          204      1111     259   1370  
                True               107           13       120      39    159  
Voluntary       False              101           28       129      24    153

## Load the data on individual inspections

In [17]:
inspections = (
    pd.read_csv(
        "../inputs/inspections-in-year-2017-18.csv",
        skiprows = 1,
        dtype = str
    )
    .assign(**{
        "Inspection date": lambda df: pd.to_datetime(df["Inspection date"], format = "%d/%m/%Y"),
        "Publication date": lambda df: pd.to_datetime(df["Publication date"], format = "%d/%m/%Y"),
    })
    [[
        "URN",
        "Provision type",
        "Registration Status",
        "Sector",
        "Event type",
        "Inspection date",
        "Publication date",
        "Overall effectiveness"
    ]]
    .merge(
        private_ownerships[[
            "URN",
            "Owner"
        ]],
        how = "left"
    )
    .assign(
        owned_by_cambian = lambda x: x["Owner"].str.contains(CAMBIAN_PATTERN, na = False, case = False),
    )
)

inspections.head()

,URN,Provision type,Registration Status,Sector,Event type,Inspection date,Publication date,Overall effectiveness,Owner,owned_by_cambian
0,1027077,Secure Training Centre,Active,Private,Full inspection,2017-10-06,2017-11-21,Inadequate,NaN,False
1,1027078,Secure Training Centre,Active,Private,Full inspection,2017-06-23,2017-08-08,Requires improvement,NaN,False
2,1027081,Residential Holiday Scheme for Disabled Children,Active,Voluntary,Full inspection,2017-08-24,2017-09-15,Good,NaN,False
3,1027082,Residential Holiday Scheme for Disabled Children,Active,Voluntary,Full inspection,2017-08-16,2017-09-11,Outstanding,NaN,False
4,1027083,Residential Holiday Scheme for Disabled Children,Active,Voluntary,Full inspection,2017-08-17,2017-09-21,Outstanding,NaN,False


### Examine the `Registration Status`, `Provision type`, `Event type`, `Overall effectiveness`, and `Sector` variables

In [18]:
inspections["Registration Status"].value_counts()

Active       3398
Resigned       84
Cancelled       6
Name: Registration Status, dtype: int64

In [19]:
inspections["Provision type"].value_counts()

Children's home                                                 2917
Residential Special School                                       151
Independent Fostering Agency                                     141
Residential special school (registered as a children's home)     118
Boarding School                                                   39
Secure children's home                                            30
Residential Family Centre                                         22
Further Education College with Residential Accommodation          18
Residential Holiday Scheme for Disabled Children                  18
Voluntary Adoption Agency                                         17
Adoption Support Agency                                           14
Secure Training Centre                                             2
CAFCASS                                                            1
Name: Provision type, dtype: int64

In [20]:
inspections["Event type"].value_counts()

Full inspection          2600
Interim inspection        580
Monitoring inspection     308
Name: Event type, dtype: int64

In [21]:
inspections["Overall effectiveness"].value_counts()

Good                               1554
Outstanding                         456
Requires improvement to be good     416
Sustained effectiveness             201
Inadequate                          175
Improved effectiveness              170
Declined in effectiveness           127
Not judged                           79
Requires improvement                  1
Name: Overall effectiveness, dtype: int64

In [22]:
inspections["Sector"].value_counts()

Private             2506
Local Authority      607
Voluntary            324
Academy               38
Health Authority      12
Public sector          1
Name: Sector, dtype: int64

## Apply data filters

In [23]:
inspections_filtered = (
    inspections
    # Only full inspections
    .loc[lambda x: x["Event type"] == "Full inspection"]

    # Filter on provision type
    .loc[lambda df: df["Provision type"].isin([
        "Children's home",
        "Residential special school (registered as a children's home)"
    ])]

    # Exclude the few homes run by health authorities
    .loc[lambda x: x["Sector"] != "Health Authority"]
)

len(inspections_filtered)

2195

In [24]:
# Make sure that we've identified all private-sector owners
assert (
    inspections_filtered
    .loc[lambda df: df["Sector"] == "Private"]
    ["Owner"].isnull()
    .sum()
) == 0

In [25]:
inspections_filtered["Overall effectiveness"].value_counts()

Good                               1349
Requires improvement to be good     363
Outstanding                         330
Inadequate                          153
Name: Overall effectiveness, dtype: int64

In [26]:
inspections_filtered["Sector"].value_counts()

Private            1630
Local Authority     404
Voluntary           161
Name: Sector, dtype: int64

## Analyze latest inspections conducted in 2017-18 inspection year

In [27]:
inspections_filtered_latest = (
    inspections_filtered
    .sort_values("Inspection date", ascending = False)
    .drop_duplicates(subset = [ "URN" ])
)

print(len(inspections_filtered_latest))

inspections_filtered_latest.head()

2089


,URN,Provision type,Registration Status,Sector,Event type,Inspection date,Publication date,Overall effectiveness,Owner,owned_by_cambian
249,1234624,Children's home,Active,Private,Full inspection,2018-03-26,2018-04-19,Good,Esland North Limited,False
3421,SC486604,Children's home,Active,Local Authority,Full inspection,2018-03-22,2018-04-19,Outstanding,NaN,False
711,1263199,Children's home,Active,Private,Full inspection,2018-03-21,2018-04-06,Good,Witherslack Group Ltd,False
719,1264016,Children's home,Active,Private,Full inspection,2018-03-21,2018-04-23,Good,Care 2 Share Limited,False
727,SC000803,Children's home,Active,Private,Full inspection,2018-03-20,2018-04-13,Good,Pear Tree Projects Limited,False


### By sector

In [28]:
(
    inspections_filtered_latest
    .groupby([
        "Sector"
    ])
    .pipe(calculate_totals)
)

Overall effectiveness,Inadequate,Requires improvement to be good,Good,Outstanding,goodplus,subpar,total
Sector,,,,,,,
Local Authority,6,60,235,91,326,66,392
Private,47,273,1008,211,1219,320,1539
Voluntary,3,23,104,28,132,26,158


### By sector and ownership

In [29]:
(
    inspections_filtered_latest
    .groupby([
        "Sector",
        "owned_by_cambian"
    ])
    .pipe(calculate_totals)
)

Overall effectiveness             Inadequate  Requires improvement to be good  \
Sector          owned_by_cambian                                                
Local Authority False                      6                               60   
Private         False                     43                              237   
                True                       4                               36   
Voluntary       False                      3                               23   

Overall effectiveness             Good  Outstanding  goodplus  subpar  total  
Sector          owned_by_cambian                                              
Local Authority False              235           91       326      66    392  
Private         False              901          199      1100     280   1380  
                True               107           12       119      40    159  
Voluntary       False              104           28       132      26    158

## Calculate proportion of homes that received a subpar evaluation in *at least one inspection*

In [30]:
def calculate_at_least_one_subpar(df, groups):
    return (
        df
        .assign(
            subpar = lambda df: (
                df
                ["Overall effectiveness"]
                .isin([
                    "Inadequate",
                    "Requires improvement to be good"
                ])
            )
        )
        .groupby([
            "URN",
        ] + groups)
        ["subpar"].max()
        .reset_index()
        .groupby(groups)
        ["subpar"].value_counts()
        .unstack()
        .assign(
            total = lambda df: df[True] + df[False]      
        )
        .assign(
            rate = lambda df: df[True] / df["total"]      
        )
    )

### By sector

In [31]:
calculate_at_least_one_subpar(
    inspections_filtered,
    [ "Sector" ]
)

subpar,False,True,total,rate
Sector,,,,
Local Authority,320,72,392,0.183673
Private,1191,348,1539,0.226121
Voluntary,131,27,158,0.170886


### By sector and ownership

In [32]:
calculate_at_least_one_subpar(
    inspections_filtered,
    [ "Sector", "owned_by_cambian" ]
)

subpar                            False  True  total      rate
Sector          owned_by_cambian                              
Local Authority False               320    72    392  0.183673
Private         False              1073   307   1380  0.222464
                True                118    41    159  0.257862
Voluntary       False               131    27    158  0.170886

---

---

---